In [1]:
import numpy as np
import cv2
import random
import os

from imutils import paths


from keras.models import Sequential
from keras.layers.core import Activation
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import load_model
from keras import backend as K
from collections import defaultdict

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, precision_recall_fscore_support, average_precision_score, confusion_matrix
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:

path='data'
data = []
labels = []
features = []


In [3]:
RANDOM_STATE = 42
IMAGE_DIM = 48
TEST_SPLIT = 0.20
BATCH_SIZE = 50
EPOCHS = 50
INIT_LR = 1e-3

In [183]:
class_dir_paths = sorted(os.listdir(path))
class_dict = {}
idx = 0

for class_dir_path in class_dir_paths:
    if (os.path.isdir(os.path.join(path, class_dir_path))):
        class_name = class_dir_path
        class_dict[class_name] = idx
        idx += 1

print("Class information : ", class_dict)

Class information :  {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6}


In [184]:
imagePaths = sorted(list(paths.list_images(path)))
random.seed(RANDOM_STATE)
random.shuffle(imagePaths)
print(paths)

<module 'imutils.paths' from '/home/tennyson/.local/lib/python3.6/site-packages/imutils/paths.py'>


In [186]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIM, IMAGE_DIM))
    image = img_to_array(image)
    feature = image_to_feature_vector(image)
    data.append(image)
    features.append(feature)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(class_dict[label])

print("Total number of images readed, ", len(data))
print("Total number of labels extracted, ", len(labels))

Total number of images readed,  1923
Total number of labels extracted,  1923


In [187]:
len(imagePaths)

1923

In [188]:
data = np.array(data, dtype="float")/255.0
labels = np.array(labels)
data.shape

(1923, 48, 48, 3)

In [189]:

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=RANDOM_STATE)

print("Number of image in training set ", len(trainX), len(trainY))
print("Number of image in tesing set ", len(testX), len(testY))

Number of image in training set  1538 1538
Number of image in tesing set  385 385


In [190]:
trainY = to_categorical(trainY, num_classes=len(class_dict))
testY = to_categorical(testY, num_classes=len(class_dict))

In [6]:

base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_DIM, IMAGE_DIM, 3))
for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [192]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

In [193]:

# Building model based on Function API
X = base_model.output
X = Flatten()(X)  

#Adding last FC layers based on hyperparams given

X = Dense(128, activation='relu')(X)
X = Dense(128, activation='relu')(X)

# for layer_param in hyperparameters['top_layers']:
# 	X = self.layers[layer_param[0]](layer_param[1], activation=layer_param[2])(X)

#Adding dropout
X = Dropout(0.5)(X)

#Adding the last layer for prediction
predictions = Dense(7, activation='softmax')(X)

#creating the final model
model = Model(base_model.input, predictions)


In [194]:

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["categorical_accuracy"])

In [195]:
model.fit(trainX, trainY, epochs=15, batch_size=10, validation_data=(testX, testY))

Train on 1538 samples, validate on 385 samples
Epoch 1/15
1538/1538 [==============================] - 63s 41ms/step - loss: 1.8112 - categorical_accuracy: 0.2555 - val_loss: 1.6374 - val_categorical_accuracy: 0.3714
Epoch 2/15
1538/1538 [==============================] - 59s 38ms/step - loss: 1.5834 - categorical_accuracy: 0.3791 - val_loss: 1.4006 - val_categorical_accuracy: 0.4494
Epoch 3/15
1538/1538 [==============================] - 59s 38ms/step - loss: 1.3816 - categorical_accuracy: 0.4642 - val_loss: 1.1992 - val_categorical_accuracy: 0.5377
Epoch 4/15
1538/1538 [==============================] - 59s 39ms/step - loss: 1.1661 - categorical_accuracy: 0.5592 - val_loss: 1.1338 - val_categorical_accuracy: 0.5688
Epoch 5/15
1538/1538 [==============================] - 63s 41ms/step - loss: 1.0034 - categorical_accuracy: 0.6216 - val_loss: 0.9826 - val_categorical_accuracy: 0.6364
Epoch 6/15
1538/1538 [==============================] - 59s 38ms/step - loss: 0.8554 - categorical_accu

In [ ]:

aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

In [196]:
# evaluate the model : train
scores = model.evaluate(trainX, trainY)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# evaluate the model : test
scores = model.evaluate(testX, testY)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1538/1538 [==============================] - 45s 30ms/step

categorical_accuracy: 93.63%
385/385 [==============================] - 11s 30ms/step

categorical_accuracy: 89.87%


In [197]:
model.save('keras_model')


In [4]:
model=load_model('keras_model')


In [12]:
camera = cv2.VideoCapture(0)
input("Show some emotion and press enter to continue")
return_value, image = camera.read()
cv2.imwrite('opencv.jpg', image)
del(camera)

Show some emotion and press enter to continue


In [15]:
image = cv2.imread('surprise.jpeg')
img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
orig = image.copy()
        
# pre-process the image for classification\n
image = cv2.resize(image, (IMAGE_DIM, IMAGE_DIM))
image = image.astype(float) / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
        # load the trained convolutional neural network

In [16]:
(anger,contempt,disgust,fear,happy,sadness,surprise) = model.predict(image)[0]
(anger,contempt,disgust,fear,happy,sadness,surprise)

(1.6636047e-09,
 2.600159e-07,
 1.0144707e-06,
 1.4753546e-05,
 1.8351139e-07,
 2.7482342e-06,
 0.99998105)

In [17]:
x=dict(zip(('anger','contempt','disgust','fear','happy','sadness','surprise'),(anger,contempt,disgust,fear,happy,sadness,surprise)))
max_key = max(x, key=lambda k: x[k])
max_key

'surprise'

In [16]:
os.remove('opencv.jpg')